In [1]:
# Yolov5 requirements
#!pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

In [2]:
# mediapipe install
#!pip install mediapipe

In [3]:
import torch

In [4]:
# torch_ver Yolov5
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s',
                            device='cuda:0' if torch.cuda.is_available() else 'cpu')  # 예측 모델
yolo_model.classes = [0]  # 예측 클래스 (0 : 사람)

Using cache found in /Users/kimsungwook/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-4-17 Python-3.11.7 torch-2.2.2 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


### 전처리 미세조정

In [7]:
start_dot = 11      # mp.solutions.pose 시작 포인트 (0: 얼굴부터 발목까지, 11: 어깨부터 발목까지)
n_CONFIDENCE = 0.3    # MediaPipe Min Detectin confidence check
y_CONFIDENCE = 0.3    # Yolv5 Min Detectin confidence check

In [8]:
import mediapipe as mp
mp_pose = mp.solutions.pose
attention_dot = [n for n in range(start_dot, 29)]

In [9]:
# 라인 그리기
if start_dot == 11:
    """몸 부분만"""
    draw_line = [[11, 13], [13, 15], [15, 21], [15, 19], [15, 17], [17, 19], \
                [12, 14], [14, 16], [16, 22], [16, 20], [16, 18], [18, 20], \
                [23, 25], [25, 27], [24, 26], [26, 28], [11, 12], [11, 23], \
                [23, 24], [12, 24]]
    print('Pose : Only Body')

else:
    """얼굴 포함"""
    draw_line = [[11, 13], [13, 15], [15, 21], [15, 19], [15, 17], [17, 19], \
                [12, 14], [14, 16], [16, 22], [16, 20], [16, 18], [18, 20], \
                [23, 25], [25, 27], [24, 26], [26, 28], [11, 12], [11, 23], \
                [23, 24], [12, 24], [9, 10], [0, 5], [0, 2], [5, 8], [2, 7]]
    print('Pose : Face + Body')

Pose : Only Body


In [11]:
import cv2

In [12]:
# Yolov4 바운딩 box 안에서 media pipe 데이터 전처리 함수

def get_skeleton(video_path, attention_dot, draw_line):
    frame_length = 30 # LSTM 모델에 넣을 frame 수

    xy_list_list, xy_list_list_flip = [], []
    cv2.destroyAllWindows()
    pose = mp_pose.Pose(static_image_mode = True, model_complexity = 1, \
                        enable_segmentation = False, min_detection_confidence = n_CONFIDENCE)
    cap = cv2.VideoCapture(video_path)

    if cap.isOpened():

        while True:
            ret, img = cap.read()

            if ret == True:

                """ Yolo 바운딩 박스 및 좌표 추출"""
                img = cv2.resize(img, (640, 640))
                res = yolo_model(img)
                res_refine = res.pandas().xyxy[0].values
                nms_human = len(res_refine)
                if nms_human > 0:
                    for bbox in res_refine:
                        """바운딩 박스 상하좌우 크기 조절"""
                        xx1, yy1, xx2, yy2 = int(bbox[0])-10, int(bbox[1]), int(bbox[2])+10, int(bbox[3])
                        if xx1 < 0:
                            xx1 = 0
                        elif xx2 > 639:
                            xx2 = 639
                        if yy1 < 0:
                            yy1 = 0
                        elif yy2 > 639:
                            yy2 = 639

                        start_point = (xx1, yy1)
                        end_point = (xx2, yy2)

                        """ Yolov5 바운딩 박스 좌표 안에서 mediapipe Pose 추출"""
                        if bbox[4] > y_CONFIDENCE: # bbox[4] : confidence 데이터
                            # img = cv2.rectangle(img, start_point, end_point, (0, 0, 255), 2) # 바운딩 박스 그리기 : 데이터 추출 확인용
                            c_img = img[yy1:yy2, xx1:xx2] # 바운딩 박스 좌표
                            results = pose.process(cv2.cvtColor(c_img, cv2.COLOR_BGR2RGB)) # Yolov5 바운딩 박스 좌표 안에서 'mp_pose' 좌표

                            if not results.pose_landmarks: continue
                            idx = 0
                            draw_line_dic = {}
                            xy_list, xy_list_flip = [], []
                            # 33 반복문 진행 : 33개 중 18개의 dot
                            for x_and_y in results.pose_landmarks.landmark:
                                if idx in attention_dot:
                                    xy_list.append(x_and_y.x)
                                    xy_list.append(x_and_y.y)
                                    xy_list_flip.append(1 - x_and_y.x)
                                    xy_list_flip.append(x_and_y.y)

                                    x, y = int(x_and_y.x*(xx2-xx1)), int(x_and_y.y*(yy2-yy1))
                                    draw_line_dic[idx] = [x, y]
                                idx += 1

                            if len(xy_list) != len(attention_dot) * 2:
                                print('Error : attention_dot 데이터 오류')

                            xy_list_list.append(xy_list)
                            xy_list_list_flip.append(xy_list_flip)

                            """mediapipe line 그리기 부분 : 데이터 추출(dot) 확인용"""
                            # for line in draw_line:
                            #     x1, y1 = draw_line_dic[line[0]][0], draw_line_dic[line[0]][1]
                            #     x2, y2 = draw_line_dic[line[1]][0], draw_line_dic[line[1]][1]
                            #     c_img = cv2.line(c_img, (x1, y1), (x2, y2), (0, 255, 0), 4)
                            # # cv2.imshow('Landmark Image', img)
                            # cv2_imshow(img)
                            # cv2.waitKey(1)

            elif ret == False: break


        # 부족한 프레임 수 맞추기
        if len(xy_list_list_flip) < 15:
            return False, False
        elif len(xy_list_list_flip) < frame_length:
            f_ln = frame_length - len(xy_list_list_flip)
            for _ in range(f_ln):
                xy_list_list.append(xy_list_list[-1])
                xy_list_list_flip.append(xy_list_list_flip[-1])

    cap.release()
    cv2.destroyAllWindows()


    return xy_list_list, xy_list_list_flip

In [13]:
video_path = '../../dataset_aihub/' # dataset 경로
video_path

'../../dataset_aihub/'

In [14]:
# 영상 데이터에서 mp pose landmark dot 데이터 추출 부분
# raw_data = []

In [15]:
# for fold in os.listdir(video_path):
#    for video_name in os.listdir(video_path + '/' + fold):
#        if int(video_name.split('_')[2][:2]) >= 30: # video name 참조
#            if video_name.split('_')[1] == 'normal': label = 0
#            else: label = 1
#            skel_data_n, skel_data_f = get_skeleton('{}/{}'.format(video_path + '/' + fold, video_name), attention_dot, draw_line)
#            if skel_data_n != False:
#                seq_list_n = skel_data_n[:30]
#                seq_list_f = skel_data_f[:30]
#                raw_data.append({'key':label, 'value':seq_list_n})
#                raw_data.append({'key':label, 'value':seq_list_f})
# random.shuffle(raw_data)

In [17]:
# 위 for문을 돌렸을 때 kernel이 죽는 오류가 발생하여 끊어서 데이터 생성

In [22]:
import pickle
import os

In [18]:
fold = 'abnormal_김성욱'
raw_data = []

In [ ]:
for video_name in os.listdir(video_path + '/' + fold):
    if int(video_name.split('_')[2][:2]) >= 30: # video name 참조
        if video_name.split('_')[1] == 'normal': label = 0
        else: label = 1
        skel_data_n, skel_data_f = get_skeleton('{}/{}'.format(video_path + '/' + fold, video_name), attention_dot, draw_line)
        if skel_data_n != False:
            seq_list_n = skel_data_n[:30]
            seq_list_f = skel_data_f[:30]
            raw_data.append({'key':label, 'value':seq_list_n})
            raw_data.append({'key':label, 'value':seq_list_f})

In [ ]:
filename = video_path + fold + '.pickle'
print(filename)
with open(filename, 'wb') as f:
    pickle.dump(raw_data, f)

In [23]:
fold = 'normal_김성욱'
raw_data = []

In [24]:
for video_name in os.listdir(video_path + '/' + fold):
    if int(video_name.split('_')[2][:2]) >= 30: # video name 참조
        if video_name.split('_')[1] == 'normal': label = 0
        else: label = 1
        skel_data_n, skel_data_f = get_skeleton('{}/{}'.format(video_path + '/' + fold, video_name), attention_dot, draw_line)
        if skel_data_n != False:
            seq_list_n = skel_data_n[:30]
            seq_list_f = skel_data_f[:30]
            raw_data.append({'key':label, 'value':seq_list_n})
            raw_data.append({'key':label, 'value':seq_list_f})

I0000 00:00:1713935712.581068   39966 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
I0000 00:00:1713935717.744036   39966 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713935721.452891   39966 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713935724.786942   39966 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713935728.861734   39966 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713935733.436328   39966 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713935736.779357   39966 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713935740.886929   39966 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713935744.868093   39966 gl_context.cc:357] GL vers

In [25]:
filename = video_path + fold + '.pickle'
print(filename)
with open(filename, 'wb') as f:
    pickle.dump(raw_data, f)

../../dataset_aihub/normal_김성욱.pickle


In [26]:
fold = 'abnormal_강민지'
raw_data = []

In [27]:
for video_name in os.listdir(video_path + '/' + fold):
    if int(video_name.split('_')[2][:2]) >= 30: # video name 참조
        if video_name.split('_')[1] == 'normal': label = 0
        else: label = 1
        skel_data_n, skel_data_f = get_skeleton('{}/{}'.format(video_path + '/' + fold, video_name), attention_dot, draw_line)
        if skel_data_n != False:
            seq_list_n = skel_data_n[:30]
            seq_list_f = skel_data_f[:30]
            raw_data.append({'key':label, 'value':seq_list_n})
            raw_data.append({'key':label, 'value':seq_list_f})

I0000 00:00:1713936749.410149   39966 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713936752.953834   39966 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713936756.765544   39966 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713936760.245355   39966 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713936763.648833   39966 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713936767.088113   39966 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713936770.510868   39966 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713936773.929019   39966 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713936777.126736   39966 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
Context le

In [28]:
filename = video_path + fold + '.pickle'
print(filename)
with open(filename, 'wb') as f:
    pickle.dump(raw_data, f)

../../dataset_aihub/abnormal_강민지.pickle


In [29]:
fold = 'normal_강민지'
raw_data = []

In [30]:
for video_name in os.listdir(video_path + '/' + fold):
    if int(video_name.split('_')[2][:2]) >= 30: # video name 참조
        if video_name.split('_')[1] == 'normal': label = 0
        else: label = 1
        skel_data_n, skel_data_f = get_skeleton('{}/{}'.format(video_path + '/' + fold, video_name), attention_dot, draw_line)
        if skel_data_n != False:
            seq_list_n = skel_data_n[:30]
            seq_list_f = skel_data_f[:30]
            raw_data.append({'key':label, 'value':seq_list_n})
            raw_data.append({'key':label, 'value':seq_list_f})

I0000 00:00:1713937418.382041   39966 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
Context leak detected, msgtracer returned -1
I0000 00:00:1713937422.176207   39966 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713937425.485588   39966 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713937428.870692   39966 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713937432.249651   39966 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713937436.383617   39966 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713937439.307677   39966 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713937442.802480   39966 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713937445.937785   39966 gl_context.cc:357] GL version: 2.1 (2

In [31]:
filename = video_path + fold + '.pickle'
print(filename)
with open(filename, 'wb') as f:
    pickle.dump(raw_data, f)

../../dataset_aihub/normal_강민지.pickle


In [32]:
import glob
pickles = sorted(glob.glob('../../dataset_aihub/*.pickle'))

In [33]:
print(pickles)

['../../dataset_aihub/abnormal_강동욱.pickle', '../../dataset_aihub/abnormal_강민지.pickle', '../../dataset_aihub/abnormal_김성욱.pickle', '../../dataset_aihub/normal_강동욱.pickle', '../../dataset_aihub/normal_강민지.pickle', '../../dataset_aihub/normal_김성욱.pickle']


In [39]:
raw_data = []
for onepickle in pickles:
    with open(onepickle, 'rb') as f:
        raw_data += pickle.load(f)

In [40]:
len(raw_data)

828

In [41]:
with open('../../dataset_aihub/raw_data.pickle', 'wb') as f:
    pickle.dump(raw_data, f)

In [36]:
for onepickle in pickles:
    with open(onepickle, 'rb') as f:
        data = pickle.load(f)
        print(onepickle,len(data))

../../dataset_aihub/abnormal_강동욱.pickle 170
../../dataset_aihub/abnormal_강민지.pickle 112
../../dataset_aihub/abnormal_김성욱.pickle 142
../../dataset_aihub/normal_강동욱.pickle 164
../../dataset_aihub/normal_강민지.pickle 108
../../dataset_aihub/normal_김성욱.pickle 132


In [42]:
with open('../../dataset_aihub/abnormal_강동욱.pickle', 'rb') as f:
    abdw = pickle.load(f)
with open('../../dataset_aihub/abnormal_강민지.pickle', 'rb') as f:
    abmj = pickle.load(f)
with open('../../dataset_aihub/abnormal_김성욱.pickle', 'rb') as f:
    absw = pickle.load(f)
with open('../../dataset_aihub/normal_강동욱.pickle', 'rb') as f:
    ndw = pickle.load(f)
with open('../../dataset_aihub/normal_강민지.pickle', 'rb') as f:
    nmj = pickle.load(f)
with open('../../dataset_aihub/normal_김성욱.pickle', 'rb') as f:
    nsw = pickle.load(f)

In [48]:
abdw = abdw[:160]

In [49]:
abmj = abmj[:108]

In [50]:
absw = absw[:132]

In [51]:
ndw = ndw[:160]

In [53]:
raw_data = abdw+abmj+absw+ndw+nmj+nsw

In [55]:
with open('../../dataset_aihub/raw_data.pickle', 'wb') as f:
    pickle.dump(raw_data, f)

In [54]:
# dataset 길이 출력

nd = 0
ad = 0
for i in range(len(raw_data)):
    if raw_data[i]['key'] == 0:
        nd += 1
    else:
        ad += 1
print('normal data:', nd, '| abnormal data:', ad)

normal data: 400 | abnormal data: 400
